# Weka Experiment

The definitive notebook.

# Preliminaries

## Imports

In [1]:
import pandas as pd
import arff
import os
import numpy as np
import aaai20
import PxW

from os.path import dirname
from aaai20.io import filename_dataset, filename_query, filename_results
from aaai20.exp import collect_results, process_outcomes, save_outcome
from sklearn.model_selection import train_test_split
from modulo.utils.encoding import query_to_code, code_to_query, encode_attribute

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

RANDOM_STATE = 42

In [2]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

## Helpers

In [3]:
def f1_weka(out, average='macro'):
    # Each label gets encoded.
    le = LabelEncoder().fit(out.values.reshape(-1,))
    
    out = out.apply(le.transform)
    f1 = f1_score(out['actual'], out['predicted'], average=average)
    return f1

## Functions

In [4]:
def fit_weka(ds,**kwargs):
    # Preliminaries
    fn_train = filename_dataset(ds, step=1, suffix='train')
    
    # Train
    clf = PxW.J48(**kwargs)
    clf.fit(fn_train, verbose=False)
    return clf

In [5]:
def predict_weka(dataset, classifier):
    result = {}
    f1_micro = []
    f1_macro = []
    
    # Get queries
    fn_qry = filename_query(dataset, suffix="default")
    q_codes = np.load(fn_qry)
    q_codes_return = q_codes.copy()
    
    for q_idx, q_code in enumerate(q_codes):
        fn = filename_dataset(dataset, step=2, suffix='q_{}'.format(str(q_idx).zfill(3)))
        
        out = clf.predict(fn, verbose=True)
        
        #print(ds)
        #print(out.head())
        
        f1_micro.append(f1_weka(out, average='micro'))
        f1_macro.append(f1_weka(out, average='macro'))
    
    result['f1_micro'] = f1_micro
    result['f1_macro'] = f1_macro
    
    return q_codes_return, result

# Actual Flow

In [6]:
datasets = ['glass',
             'credit-g',
             'ionosphere',
             'lymph',
             'vehicle',
             'iris',
             'splice',
             'sonar',
             'vowel',
             'segment',
             'zoo',
             'heart-statlog',
             'waveform-5000',
             'kr-vs-kp',
             'diabetes',
             'letter',
             'balance-scale']

MAX_DEPTH = 8
NB_ITERATIONS = 2

print(len(datasets))

dataframes = {k:[] for k in datasets}

for algorithm in {"J48", "RF"}:
    for ds in datasets:
        identifier = algorithm
        
        # Get nb_atts
        fn_qry = filename_query(ds, suffix="default")
        q_codes = np.load(fn_qry)
        nb_atts = q_codes.shape[1]
        
        nb_estimators = nb_atts*NB_ITERATIONS
        print("""
        For dataset {}, we learn {} trees.
        """.format(ds, nb_estimators))
        
        # do stuff
        clf = fit_weka(ds, algorithm=algorithm, max_depth=MAX_DEPTH, nb_estimators=nb_estimators)

        q_codes, results = predict_weka(ds, clf)

        dataframes[ds] = collect_results(ds, q_codes, results, identifier=algorithm)

    df = process_outcomes(dataframes)

    fn = filename_results(exp_dname='mercs-vs-weka', exp_fname=algorithm)
    df.to_csv(fn)
   

[2019-10-15 15:14:49,872] INFO - prefect.FlowRunner | Beginning Flow run for 'fit'
[2019-10-15 15:14:49,873] INFO - prefect.FlowRunner | Starting flow run.
[2019-10-15 15:14:49,877] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-10-15 15:14:49,878] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-10-15 15:14:49,879] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...


17
java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar weka.classifiers.trees.J48 -C 0.25 -M 2 -no-cv -t /cw/dtailocal/repos/aaai20/data/step-01/glass-train.arff -d tree.model


[2019-10-15 15:14:50,315] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-10-15 15:14:50,319] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-10-15 15:14:50,322] INFO - prefect.FlowRunner | Beginning Flow run for 'predict'
[2019-10-15 15:14:50,323] INFO - prefect.FlowRunner | Starting flow run.
[2019-10-15 15:14:50,329] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-10-15 15:14:50,331] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-10-15 15:14:50,332] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-10-15 15:14:50,554] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-10-15 15:14:50,555] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-10-15 15:14:50,571] INFO - prefect.FlowRunner | Beginni

java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar weka.classifiers.trees.J48 -C 0.25 -M 2 -no-cv -t /cw/dtailocal/repos/aaai20/data/step-01/credit-g-train.arff -d tree.model


[2019-10-15 15:14:52,917] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-10-15 15:14:52,919] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-10-15 15:14:52,923] INFO - prefect.FlowRunner | Beginning Flow run for 'predict'
[2019-10-15 15:14:52,925] INFO - prefect.FlowRunner | Starting flow run.
[2019-10-15 15:14:52,931] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-10-15 15:14:52,933] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-10-15 15:14:52,935] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-10-15 15:14:53,255] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-10-15 15:14:53,256] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-10-15 15:14:53,266] INFO - prefect.FlowRunner | Beginni

...

[2019-10-15 15:18:34,927] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-10-15 15:18:34,928] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-10-15 15:18:34,930] INFO - prefect.FlowRunner | Beginning Flow run for 'predict'
[2019-10-15 15:18:34,931] INFO - prefect.FlowRunner | Starting flow run.
[2019-10-15 15:18:34,935] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-10-15 15:18:34,936] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-10-15 15:18:34,937] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-10-15 15:18:35,255] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-10-15 15:18:35,256] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-10-15 15:18:35,272] INFO - prefect.FlowRunner | Beginni

java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar weka.classifiers.trees.RandomForest -I 34 -depth 8 -no-cv -t /cw/dtailocal/repos/aaai20/data/step-01/letter-train.arff -d tree.model


[2019-10-15 15:18:40,511] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-10-15 15:18:40,512] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-10-15 15:18:40,514] INFO - prefect.FlowRunner | Beginning Flow run for 'predict'
[2019-10-15 15:18:40,516] INFO - prefect.FlowRunner | Starting flow run.
[2019-10-15 15:18:40,520] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-10-15 15:18:40,521] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-10-15 15:18:40,521] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-10-15 15:18:41,008] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-10-15 15:18:41,009] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-10-15 15:18:41,026] INFO - prefect.FlowRunner | Beginni

java -cp /home/elia/Software/weka/weka-3-8-3/weka.jar weka.classifiers.trees.RandomForest -I 10 -depth 8 -no-cv -t /cw/dtailocal/repos/aaai20/data/step-01/balance-scale-train.arff -d tree.model


[2019-10-15 15:19:06,499] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-10-15 15:19:06,500] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-10-15 15:19:06,502] INFO - prefect.FlowRunner | Beginning Flow run for 'predict'
[2019-10-15 15:19:06,503] INFO - prefect.FlowRunner | Starting flow run.
[2019-10-15 15:19:06,506] INFO - prefect.TaskRunner | Task 'Constant[str]': Starting task run...
[2019-10-15 15:19:06,507] INFO - prefect.TaskRunner | Task 'Constant[str]': finished task run for task with final state: 'Success'
[2019-10-15 15:19:06,508] INFO - prefect.TaskRunner | Task 'ShellTask': Starting task run...
[2019-10-15 15:19:06,812] INFO - prefect.TaskRunner | Task 'ShellTask': finished task run for task with final state: 'Success'
[2019-10-15 15:19:06,814] INFO - prefect.FlowRunner | Flow run SUCCESS: all reference tasks succeeded
[2019-10-15 15:19:06,826] INFO - prefect.FlowRunner | Beginni